In [10]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate

In [19]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and memory =1
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
# df.i=df.i.astype(str)+"%"
# df.fp=df.fp.astype(str)+"%"
# df.alive=df.alive.astype(str)+"%"
# df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total memory','i':'intentional memory','fp': 'false positive memory','c':'unknown elimination memory','alive':'alive memory'})
# df=df.rename(columns={'total':'Total Memory Alerts','i':'Intentional','fp': 'False Positive','c':'Unknown Elimination','alive':'Alive'})
memory=df
print(tabulate(memory, tablefmt='pipe',headers='keys',showindex=False))

| name         |   total memory |   intentional memory |   false positive memory |   alive memory |   unknown elimination memory |
|:-------------|---------------:|---------------------:|------------------------:|---------------:|-----------------------------:|
| Chromium EC  |            306 |                 0.33 |                    2.61 |           3.59 |                        63.73 |
| Firefox      |           5448 |                 3.82 |                    9.69 |           8.87 |                        25.11 |
| Kodi         |            682 |                10.56 |                    9.38 |          11.44 |                         9.09 |
| LibreOffice  |           4194 |                 0.81 |                   18.36 |           0.07 |                         6.75 |
| Linux        |           7903 |                 1.64 |                    4.02 |          21.08 |                        28.22 |
| OpenCV       |            710 |                 0.56 |                    1.97 | 

In [20]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and memory is null
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
# df.i=df.i.astype(str)+"%"
# df.fp=df.fp.astype(str)+"%"
# df.alive=df.alive.astype(str)+"%"
# df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total nonmemory','i':'intentional nonmemory','fp': 'false positive nonmemory','c':'unknown elimination nonmemory','alive':'alive nonmemory'})
nonmemory=df
nonmemory

,name,total nonmemory,intentional nonmemory,false positive nonmemory,alive nonmemory,unknown elimination nonmemory
0,Chromium EC,658,8.66,9.57,9.88,43.16
1,Firefox,7497,10.32,5.80,12.47,22.28
2,Kodi,1711,15.08,1.17,16.42,14.03
3,LibreOffice,8321,3.68,2.20,0.05,22.82
4,Linux,11611,5.66,3.81,24.92,22.77
5,OpenCV,1733,7.04,2.71,13.79,33.18
6,Samba,2807,2.21,2.24,21.80,47.17
7,Thunderbird,528,0.57,0.19,46.59,34.85
8,VTK,1356,0.44,1.77,77.88,9.96
9,ovirt-engine,1717,9.26,38.61,4.19,13.92


In [21]:
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[memory,nonmemory])
df=df.reset_index()
df

,index,name,total memory,intentional memory,false positive memory,alive memory,unknown elimination memory,total nonmemory,intentional nonmemory,false positive nonmemory,alive nonmemory,unknown elimination nonmemory
0,0,Chromium EC,306,0.33,2.61,3.59,63.73,658,8.66,9.57,9.88,43.16
1,1,Firefox,5448,3.82,9.69,8.87,25.11,7497,10.32,5.80,12.47,22.28
2,2,Kodi,682,10.56,9.38,11.44,9.09,1711,15.08,1.17,16.42,14.03
3,3,LibreOffice,4194,0.81,18.36,0.07,6.75,8321,3.68,2.20,0.05,22.82
4,4,Linux,7903,1.64,4.02,21.08,28.22,11611,5.66,3.81,24.92,22.77
5,5,OpenCV,710,0.56,1.97,8.31,41.55,1733,7.04,2.71,13.79,33.18
6,6,Samba,1700,0.82,2.88,18.12,35.53,2807,2.21,2.24,21.80,47.17
7,7,Thunderbird,357,0.28,0.28,31.37,38.94,528,0.57,0.19,46.59,34.85
8,8,VTK,1166,0.09,0.17,86.96,4.80,1356,0.44,1.77,77.88,9.96
9,9,ovirt-engine,1272,5.58,49.21,9.04,8.57,1717,9.26,38.61,4.19,13.92


In [24]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['intentional memory'], df['intentional nonmemory'])
stat, p, df['intentional memory'].median(), df['intentional nonmemory'].median()

(24.0, 0.026951278584693587, 0.815, 6.35)

In [25]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['false positive memory'], df['false positive nonmemory'])
stat, p, df['false positive memory'].median(), df['false positive nonmemory'].median()

(42.0, 0.28537519402908695, 3.4499999999999997, 2.475)

In [26]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['alive memory'], df['alive nonmemory'])
stat, p, df['alive memory'].median(), df['alive nonmemory'].median()

(41.0, 0.26026144163788634, 10.239999999999998, 15.105)

In [13]:
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='Intentional'
    and memory = 1
    group by name
    order by name; '''
tma=pd.DataFrame(sql.execute(q))
tma

,name,memory
0,Chromium EC,1
1,Firefox,208
2,Kodi,72
3,LibreOffice,34
4,Linux,130
5,OpenCV,4
6,ovirt-engine,71
7,Samba,14
8,Thunderbird,1
9,VTK,1


In [14]:
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='Intentional'
    and memory is null
    group by name
    order by name; '''
tnma=pd.DataFrame(sql.execute(q))
tnma

,name,non_memory
0,Chromium EC,57
1,Firefox,774
2,Kodi,258
3,LibreOffice,306
4,Linux,657
5,OpenCV,122
6,ovirt-engine,159
7,Samba,62
8,Thunderbird,3
9,VTK,6


In [15]:
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'intentional'})
df

NameError: name 'ta' is not defined

In [16]:
q='''select name, count(*) as total from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    group by name
    order by name; '''
ta=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    and memory = 1
    group by name
    order by name; '''
tma=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    where is_invalid = 0
    and classification ='False Positive'
    and memory is null
    group by name
    order by name; '''
tnma=pd.DataFrame(sql.execute(q))
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'false positive'})
df

,name,false positive,memory,non_memory
0,!CHAOS Control System,4,75.0%,25.0%
1,Chromium EC,71,11.27%,88.73%
2,Firefox,963,54.83%,45.17%
3,Kodi,84,76.19%,23.81%
4,LibreOffice,953,80.8%,19.2%
5,Linux,760,41.84%,58.16%
6,OpenCV,61,22.95%,77.05%
7,ovirt-engine,1289,48.56%,51.44%
8,Samba,112,43.75%,56.25%
9,Thunderbird,2,50.0%,50.0%


In [17]:
q='''select name, count(*) as total from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
ta=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    and me.memory=1
    group by name
    order by name;
    '''
tma=pd.DataFrame(sql.execute(q))
q='''select name, count(*) as non_memory from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    and me.memory is null
    group by name
    order by name;
    '''
tnma=pd.DataFrame(sql.execute(q))
df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])
df.memory= round((df.memory/df.total) * 100,2)
df.non_memory= round((df.non_memory/df.total) * 100,2)
df['memory']=df['memory'].astype(str)+"%"
df['non_memory']=df['non_memory'].astype(str)+"%"
df=df.rename(columns={'total':'elimination unknown'})
df

,name,elimination unknown,memory,non_memory
0,!CHAOS Control System,1355,8.12%,91.88%
1,Chromium EC,479,40.71%,59.29%
2,Firefox,3038,45.03%,54.97%
3,Kodi,302,20.53%,79.47%
4,LibreOffice,2182,12.97%,87.03%
5,Linux,4874,45.75%,54.25%
6,OpenCV,870,33.91%,66.09%
7,ovirt-engine,348,31.32%,68.68%
8,Samba,1928,31.33%,68.67%
9,Thunderbird,323,43.03%,56.97%
